# Libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)

D:\Development\Anaconda\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
from nba_api.stats.static import players as nba_players
from nba_api.stats.endpoints import commonplayerinfo

import time

# Data

In [3]:
games = pd.read_csv('./data/games.csv')
games_details = pd.read_csv('./data/games_details.csv')

players = pd.read_csv('./data/players.csv')

teams = pd.read_csv('./data/teams.csv')
ranking = pd.read_csv('./data/ranking.csv')

In [4]:
games['GAME_DATE_EST'] = pd.to_datetime(games['GAME_DATE_EST'], format='%Y-%m-%d')

## GAMES DETAILS

In [5]:
df = pd.merge(games_details, games[['GAME_ID', 'SEASON', 'GAME_DATE_EST']], on='GAME_ID', how='inner')
df = df.sort_values('GAME_DATE_EST')

df['MIN'] = pd.to_numeric(df['MIN'].str.strip(':').str[0:2], errors='coerce')

In [6]:
last_n_games = [5, 10]
columns_to_agg = ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 
                  'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 
                  'STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS']

for game in last_n_games:
    for col in columns_to_agg:
        df[f'AVG_{col}_LAST_{game}'] = df.groupby(['SEASON', 'PLAYER_ID'])[col].shift(1).rolling(game, min_periods=1).mean()
        
    df[f'GAMES_PLAYED_LAST_{game}'] = df.groupby(['SEASON', 'PLAYER_ID'])['MIN'].shift(1).rolling(game + 1, min_periods=1).count()/game

In [7]:
# df.tail()

## GAMES

In [8]:
col_home = ['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID','HOME_TEAM_ID','SEASON', 
            'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 
            'AST_home', 'REB_home', 'HOME_TEAM_WINS']

col_guest = ['GAME_DATE_EST', 'GAME_ID','HOME_TEAM_ID','VISITOR_TEAM_ID', 'SEASON', 
            'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 
            'AST_away', 'REB_away', 'HOME_TEAM_WINS']

col_names = ['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'TEAM_ID', 'SEASON', 
            'PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 
            'AST', 'REB', 'TEAM_WINS']

games_home = games[col_home]
games_home.columns = col_names
games_home = games_home.sort_values('GAME_DATE_EST')

games_guest = games[col_guest]
games_guest['HOME_TEAM_WINS'] = 1 - games['HOME_TEAM_WINS']
games_guest.columns = col_names
games_guest = games_guest.sort_values('GAME_DATE_EST')

all_games = pd.concat([games_home, games_guest])
all_games = all_games.sort_values('GAME_DATE_EST')

In [9]:
last_n_games = [5, 10]
columns_to_agg = ['PTS','FG_PCT','FT_PCT','FG3_PCT',
                  'AST','REB','TEAM_WINS']
for game in last_n_games:
    for col in columns_to_agg:
        all_games[f'TEAM_AVG_{col}_LAST_{game}'] = all_games.groupby(['SEASON', 'TEAM_ID'])[col].shift(1).rolling(game, min_periods=1).mean()

In [10]:
all_games_cols = ['GAME_ID', 'TEAM_ID', 'TEAM_AVG_PTS_LAST_5', 'TEAM_AVG_FG_PCT_LAST_5', 'TEAM_AVG_FT_PCT_LAST_5', 
                  'TEAM_AVG_FG3_PCT_LAST_5', 'TEAM_AVG_AST_LAST_5', 'TEAM_AVG_REB_LAST_5',
                  'TEAM_AVG_TEAM_WINS_LAST_5', 'TEAM_AVG_PTS_LAST_10', 'TEAM_AVG_FG_PCT_LAST_10', 
                  'TEAM_AVG_FT_PCT_LAST_10', 'TEAM_AVG_FG3_PCT_LAST_10', 'TEAM_AVG_AST_LAST_10',
                  'TEAM_AVG_REB_LAST_10', 'TEAM_AVG_TEAM_WINS_LAST_10']

all_games = all_games[all_games_cols]

In [11]:
# all_games.tail()

## NIGHTLIFE

In [12]:
nightlife = pd.read_excel('./data/nightlife.xlsx')

nightlife = nightlife[['Abbrev', 'AVERAGE SC RATING']]

## PLAYERS

In [13]:
# positions = []
# for index, player in players.iterrows():
#     print(player['PLAYER_NAME'])
#     player_info = nba_players.find_players_by_full_name(player['PLAYER_NAME'])
#     player_info = commonplayerinfo.CommonPlayerInfo(player_id = player_info[0]['id'])
    
#     position = player_info.get_normalized_dict()['CommonPlayerInfo'][0]['POSITION']
#     positions.append(position)
    
#     time.sleep(2) # ALLOWABLE NUMBER OF CONNECTION IS 30 PER MINUTE

# players['POSITION'] = positions

## Merge dataframes

In [14]:
df = df.merge(all_games, on=['GAME_ID', 'TEAM_ID'], how='inner') #MERGE WITH GAMES
df = df.merge(nightlife, left_on='TEAM_ABBREVIATION', right_on='Abbrev', how='inner') # GET NIGHTLIFE QUALITY
# df = pd.merge(df, players, on='PLAYER_ID', how='inner') #MERGE WITH PLAYERS TO GET POSITION

## REDUCE DATAFRAME

In [15]:
df_reduced = df.set_index(['PLAYER_ID', 'SEASON']).loc[df.groupby(['PLAYER_ID', 'SEASON'])['MIN'].mean() > 20, :] # THOSE WHO PLAYER OVER 20 MIN PER GAME
df_reduced = df_reduced.loc[(df_reduced['AVG_MIN_LAST_5'] >= 0) & (df_reduced['MIN'] >= 0), :] # SELECT ONLY WHEN PLAYER PLAYED A GAME (AND CONSEQUENTLY REMOVE THE FIRST GAME OF THE SEASON)

In [16]:
# JUST FOR CONVENIENCE
df_reduced = df_reduced.reset_index()
df_reduced = df_reduced.set_index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION', 'SEASON'])

## GENERATE NEW FEATURES

In [17]:
interesting_ratios = [['AVG_PTS_LAST_5', 'TEAM_AVG_PTS_LAST_5'], 
                      ['AVG_AST_LAST_5', 'TEAM_AVG_AST_LAST_5'], 
                      ['AVG_REB_LAST_5', 'TEAM_AVG_REB_LAST_5'],
                      ['AVG_PTS_LAST_10', 'TEAM_AVG_PTS_LAST_10'], 
                      ['AVG_AST_LAST_10', 'TEAM_AVG_AST_LAST_10'], 
                      ['AVG_REB_LAST_10', 'TEAM_AVG_REB_LAST_10']]

for ratio in interesting_ratios:
    df_reduced[f'RATIO_{ratio[0]}_{ratio[1]}'] = df_reduced[ratio[0]]/df_reduced[ratio[1]]

In [18]:
columns_X = ['AVG_MIN_LAST_5', 'AVG_FGM_LAST_5',
       'AVG_FGA_LAST_5', 'AVG_FG_PCT_LAST_5', 'AVG_FG3M_LAST_5',
       'AVG_FG3A_LAST_5', 'AVG_FG3_PCT_LAST_5', 'AVG_FTM_LAST_5',
       'AVG_FTA_LAST_5', 'AVG_FT_PCT_LAST_5', 'AVG_OREB_LAST_5',
       'AVG_DREB_LAST_5', 'AVG_REB_LAST_5', 'AVG_AST_LAST_5', 'AVG_STL_LAST_5',
       'AVG_BLK_LAST_5', 'AVG_TO_LAST_5', 'AVG_PF_LAST_5', 'AVG_PTS_LAST_5',
       'AVG_PLUS_MINUS_LAST_5', 'GAMES_PLAYED_LAST_5', 'AVG_MIN_LAST_10',
       'AVG_FGM_LAST_10', 'AVG_FGA_LAST_10', 'AVG_FG_PCT_LAST_10',
       'AVG_FG3M_LAST_10', 'AVG_FG3A_LAST_10', 'AVG_FG3_PCT_LAST_10',
       'AVG_FTM_LAST_10', 'AVG_FTA_LAST_10', 'AVG_FT_PCT_LAST_10',
       'AVG_OREB_LAST_10', 'AVG_DREB_LAST_10', 'AVG_REB_LAST_10',
       'AVG_AST_LAST_10', 'AVG_STL_LAST_10', 'AVG_BLK_LAST_10',
       'AVG_TO_LAST_10', 'AVG_PF_LAST_10', 'AVG_PTS_LAST_10',
       'AVG_PLUS_MINUS_LAST_10', 'GAMES_PLAYED_LAST_10', 'TEAM_AVG_PTS_LAST_5',
       'TEAM_AVG_FG_PCT_LAST_5', 'TEAM_AVG_FT_PCT_LAST_5',
       'TEAM_AVG_FG3_PCT_LAST_5', 'TEAM_AVG_AST_LAST_5', 'TEAM_AVG_REB_LAST_5',
       'TEAM_AVG_TEAM_WINS_LAST_5', 'TEAM_AVG_PTS_LAST_10',
       'TEAM_AVG_FG_PCT_LAST_10', 'TEAM_AVG_FT_PCT_LAST_10',
       'TEAM_AVG_FG3_PCT_LAST_10', 'TEAM_AVG_AST_LAST_10',
       'TEAM_AVG_REB_LAST_10', 'TEAM_AVG_TEAM_WINS_LAST_10',
       'RATIO_AVG_PTS_LAST_5_TEAM_AVG_PTS_LAST_5',
       'RATIO_AVG_AST_LAST_5_TEAM_AVG_AST_LAST_5',
       'RATIO_AVG_REB_LAST_5_TEAM_AVG_REB_LAST_5',
       'RATIO_AVG_PTS_LAST_10_TEAM_AVG_PTS_LAST_10',
       'RATIO_AVG_AST_LAST_10_TEAM_AVG_AST_LAST_10',
       'RATIO_AVG_REB_LAST_10_TEAM_AVG_REB_LAST_10',
       'AVERAGE SC RATING']
columns_Y = ['PTS', 'REB', 'AST']

df_reduced = df_reduced[columns_X + columns_Y]

In [19]:
df_reduced.to_csv('Prepared Data/df.csv')